In [1]:
import pandas as pd
df_qu = pd.read_csv('coursera_ques.csv').set_index('index')
posts = pd.read_csv("Posts.csv",  encoding = "ISO-8859-1").dropna(subset=['Tags'])[['Id', 'Tags']].values

/home/yas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/yas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
df_qu.head()

,Text
index,
https://www.coursera.org/learn/python,cours aim teach everyon basic program comput u...
https://www.coursera.org/learn/algorithms-part1,cours cover essenti inform everi seriou progra...
https://www.coursera.org/learn/python-data-analysis,cours introduc learner basic python program en...
https://www.coursera.org/learn/algorithmic-toolbox,cours cover basic algorithm techniqu idea comp...
https://www.coursera.org/learn/python-data,cours introduc core data structur python progr...


In [3]:
from sklearn.neighbors import NearestNeighbors

In [4]:
idx = 533

In [5]:
courses = df_qu.values[:idx,:]
questions = df_qu.values[idx:, :]

In [6]:
tags_df = pd.read_csv('Tags.csv',  encoding = "ISO-8859-1" )
tags = tags_df.sort_values('Count', ascending=False)['TagName'].values[:500]

In [7]:
tags.shape

(500,)

In [8]:
tags_indexes = dict([(tag, index) for index, tag in enumerate(tags)])

In [9]:
import numpy as np
from scipy.sparse import csr_matrix

In [10]:
array = np.zeros((courses.shape[0],tags.shape[0]*2) , dtype=np.int8)
for i in range(courses.shape[0]):
    for j in range(tags.shape[0]):
        if tags[j] in courses[i, 1]:
            array[i, tags_indexes[tags[j]]] = 1
            array[i, tags_indexes[tags[j]] + tags.shape[0]] = 1

In [11]:
courses_df = pd.DataFrame(array)
courses_df['id'] = courses[:,0]
courses_df.set_index('id',inplace=True)

In [12]:
array1 = np.zeros((questions.shape[0],tags.shape[0]*2) , dtype=np.int8)
for i in range(questions.shape[0]):
    for j in range(tags.shape[0]):
        if tags[j] in questions[i, 1]:
            array1[i, tags_indexes[tags[j]]] = 1
        if tags[j] in posts[i, 1]:
            array1[i, tags_indexes[tags[j]] + tags.shape[0]] = 1

In [13]:
questions_df = pd.DataFrame(array1)
questions_df['id'] = questions[:,0]
questions_df.set_index('id',inplace=True)

In [14]:
union = csr_matrix(pd.concat([courses_df, questions_df],axis=0))
questions_df = csr_matrix(questions_df)
courses_df = csr_matrix(courses_df)

In [15]:
tags = None
tags_indexes = None
array = None
array1 = None

In [16]:
import gc
gc.collect()

28

In [29]:
nbrs = NearestNeighbors(n_neighbors=10).fit(union)
nbrs_courses = NearestNeighbors(n_neighbors=10).fit(courses_df)
nbrs_questions = NearestNeighbors(n_neighbors=10).fit(questions_df)

In [23]:
def get_closest_neighs(name, target='course'):
    if target == 'course':
        nbrs_local = nbrs_courses
    elif target == 'question':
        nbrs_local = nbrs_questions
    elif target == 'multi':
        nbrs_local = nbrs
    row = df_qu.index.get_loc(name)
    distances, indices = nbrs_local.kneighbors(union.getrow(row))
    indexes_similar = pd.Series(indices.flatten()).map(df_qu.reset_index()['index'])
    result = pd.DataFrame({'distance': distances.flatten(), 'index': indexes_similar})
    
    return result

In [26]:
pred = get_closest_neighs("44698627", "course")

In [30]:
pred

,distance,index
0,3.872983,https://www.coursera.org/learn/galaxy-project
1,3.872983,https://www.coursera.org/learn/cyber-security-...
2,4.123106,https://www.coursera.org/learn/ui-design-capstone
3,4.123106,https://www.coursera.org/learn/cloud-computing...
4,4.123106,https://www.coursera.org/learn/digital-democracy
5,4.123106,https://www.coursera.org/learn/wireless-commun...
6,4.358899,https://www.coursera.org/learn/game-design
7,4.358899,https://www.coursera.org/learn/recommender-met...
8,4.358899,https://www.coursera.org/learn/android-program...
9,4.582576,https://www.coursera.org/learn/r-capstone


4

{'author',
 'ca',
 'case',
 'client',
 'com',
 'confirm',
 'design',
 'get',
 'like',
 'ssl',
 'string',
 'system',
 'text',
 'url'}